# RIGR: Resonance Invariant Graph Representation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/chemprop/chemprop/blob/main/examples/rigr_featurizer.ipynb)

In [22]:
# Install chemprop from GitHub if running in Google Colab
import os

if os.getenv("COLAB_RELEASE_TAG"):
    try:
        import chemprop
    except ImportError:
        !git clone https://github.com/chemprop/chemprop.git
        %cd chemprop
        !pip install .
        %cd examples

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from lightning import pytorch as pl
from typing import Sequence
from rdkit.Chem.rdchem import Atom, Bond, Mol
from rdkit import Chem

from chemprop import data, featurizers, models, nn
from chemprop.featurizers.atom import RIGRAtomFeaturizer
from chemprop.featurizers.bond import RIGRBondFeaturizer
from chemprop.featurizers.molecule import ChargeFeaturizer
from chemprop.utils import make_mol

In [4]:
chemprop_dir = Path.cwd().parent
input_path = chemprop_dir / "tests" / "data" / "regression" / "mol" / "mol.csv" # path to your data .csv file
num_workers = 0 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'smiles' # name of the column containing SMILES strings
target_columns = ['lipo'] # list of names of the columns containing targets

In [5]:
df_input = pd.read_csv(input_path)
df_input

,smiles,lipo
0,Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14,3.54
1,COc1cc(OC)c(cc1NC(=O)CSCC(=O)O)S(=O)(=O)N2C(C)...,-1.18
2,COC(=O)[C@@H](N1CCc2sccc2C1)c3ccccc3Cl,3.69
3,OC[C@H](O)CN1C(=O)C(Cc2ccccc12)NC(=O)c3cc4cc(C...,3.37
4,Cc1cccc(C[C@H](NC(=O)c2cc(nn2C)C(C)(C)C)C(=O)N...,3.10
...,...,...
95,CC(C)N(CCCNC(=O)Nc1ccc(cc1)C(C)(C)C)C[C@H]2O[C...,2.20
96,CCN(CC)CCCCNc1ncc2CN(C(=O)N(Cc3cccc(NC(=O)C=C)...,2.04
97,CCSc1c(Cc2ccccc2C(F)(F)F)sc3N(CC(C)C)C(=O)N(C)...,4.49
98,COc1ccc(Cc2c(N)n[nH]c2N)cc1,0.20


In [6]:
smis = df_input.loc[:, smiles_column].values
ys = df_input.loc[:, target_columns].values

## Featurization and Make Dataset

In [8]:
mols = [make_mol(smi, add_h=True, keep_h=True) for smi in smis]

charge_featurizer = ChargeFeaturizer()
charge_feats = [charge_featurizer(mol) for mol in mols]
x_ds = [np.hstack([charge_feat]) for charge_feat in zip(charge_feats)]

all_data = [data.MoleculeDatapoint(mol, name=smi, y=y, x_d=x_d) for mol, smi, y, x_d in zip(mols, smis, ys, x_ds)]
train_indices, val_indices, test_indices = data.make_split_indices(mols, "random", (0.8, 0.1, 0.1))
train_data, val_data, test_data = data.split_data_by_indices(
    all_data, train_indices, val_indices, test_indices
)

The return type of make_split_indices has changed in v2.1 - see help(make_split_indices)


In [10]:
train_data

[[MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x7fcb8d15c270>, y=array([3.59]), weight=1.0, gt_mask=None, lt_mask=None, x_d=array([[0]]), x_phase=None, name='CC(C#C)N1C(=O)N(CC2CC2)c3nn(Cc4ccnc5ccc(Cl)cc45)c(c3C1=O)c6cncn6C', V_f=None, E_f=None, V_d=None),
  MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x7fcb8d639850>, y=array([3.69]), weight=1.0, gt_mask=None, lt_mask=None, x_d=array([[0]]), x_phase=None, name='COC(=O)[C@@H](N1CCc2sccc2C1)c3ccccc3Cl', V_f=None, E_f=None, V_d=None),
  MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x7fcb8d63b4c0>, y=array([3.8]), weight=1.0, gt_mask=None, lt_mask=None, x_d=array([[0]]), x_phase=None, name='N(c1ccccc1)c2cc(Nc3ccccc3)[nH]n2', V_f=None, E_f=None, V_d=None),
  MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x7fcb8d6393f0>, y=array([0.2]), weight=1.0, gt_mask=None, lt_mask=None, x_d=array([[0]]), x_phase=None, name='COc1cc(NS(=O)(=O)c2ccc(N)cc2)nc(OC)n1', V_f=None, E_f=None, V_d=None),
  MoleculeDatapoin

In [12]:
rigr_atom_featurizer = RIGRAtomFeaturizer()
rigr_bond_featurizer = RIGRBondFeaturizer()
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer(atom_featurizer=rigr_atom_featurizer, bond_featurizer=rigr_bond_featurizer)

train_dset = data.MoleculeDataset(train_data[0], featurizer)
scaler = train_dset.normalize_targets()

val_dset = data.MoleculeDataset(val_data[0], featurizer)
val_dset.normalize_targets(scaler)

test_dset = data.MoleculeDataset(test_data[0], featurizer)

In [13]:
train_loader = data.build_dataloader(train_dset, num_workers=num_workers)
val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)
test_loader = data.build_dataloader(test_dset, num_workers=num_workers, shuffle=False)

In [15]:
mp = nn.BondMessagePassing(
    d_v=featurizer.atom_fdim,
    d_e=featurizer.bond_fdim,
)
agg = nn.MeanAggregation()
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
ffn = nn.RegressionFFN(
    input_dim=mp.output_dim + train_dset.d_xd,
    output_transform=output_transform,
)
batch_norm = True
metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()] # Only the first metric is used for training and early stopping
mpnn = models.MPNN(mp, agg, ffn, batch_norm, metric_list)
mpnn

MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=54, out_features=300, bias=False)
    (W_h): Linear(in_features=300, out_features=300, bias=False)
    (W_o): Linear(in_features=352, out_features=300, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): MeanAggregation()
  (bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=301, out_features=300, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=300, out_features=1, bias=True)
      )
    )
    (criterion): MSE(task_weights=[[1.0]])
    (output_transform): UnscaleTransform()
  )
  (X_d_transform): Identity()
  (metrics): ModuleList(
    (0): RMSE(task_weights=[[1.0]])
    (1): MAE(task_weigh

In [16]:
trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=20, # number of epochs to train for
)

/home/akshatz/anaconda3/envs/chemprop/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/akshatz/anaconda3/envs/chemprop/lib/python3.12 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


## Start Training

In [19]:
trainer.fit(mpnn, train_loader, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 212 K  | eval 
1 | agg             | MeanAggregation    | 0      | eval 
2 | bn              | BatchNorm1d        | 600    | eval 
3 | predictor       | RegressionFFN      | 90.9 K | eval 
4 | X_d_transform   | Identity           | 0      | train
5 | metrics         | ModuleList         | 0      | eval 
---------------------------------------------------------------
303 K     Trainable params
0         Non-trainable params
303 K     Total params
1.214     Total estimated model params size (MB)
4         Modules in train mode
21        Modules in eval mode


RuntimeError: Tensors must have same number of dimensions: got 2 and 3

## Test Results

In [21]:
results = trainer.test(mpnn, test_loader)

/home/akshatz/anaconda3/envs/chemprop/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/akshatz/anaconda3/envs/chemprop/lib/python3.12 ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/akshatz/anaconda3/envs/chemprop/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

RuntimeError: Tensors must have same number of dimensions: got 2 and 3